In [0]:
pip install reverse-geocode

     |████████████████████████████████| 1.6MB 3.4MB/s 
  Created wheel for reverse-geocode: filename=reverse_geocode-1.4-cp36-none-any.whl size=1637801 sha256=9044070f54136e57b9f9d58a9055ef03e574e5cfe3d3f17ba643d9dfda13e2c3
  Stored in directory: /root/.cache/pip/wheels/80/56/82/529b8b1d91c1ef49fe335d89d61bc4d244b6ed72c5063c55e9
Successfully built reverse-geocode


In [0]:
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np  #linear algebra
import re           #regular expressions
import geopy.distance as distance
import math
import timeit
import os
import reverse_geocode

In [0]:
pd.options.display.max_rows = 100

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#reading data
data = pd.read_csv("drive/My Drive/FYP/Data/ads_fact_500000.csv", engine='python', encoding='utf-8', error_bad_lines=False)

In [0]:
print(data)

           ad_id  make_id  model_id  ...  duplicate  click_count  ad_status
0       73919640       34      1500  ...          0            0          0
1       73919652        8      1519  ...          0            0          0
2       73919677        8      1519  ...          0            0          0
3       73919705       66      1547  ...          0            0          0
4       73919723       81      1514  ...          0            0          0
...          ...      ...       ...  ...        ...          ...        ...
499995  93031487       86      1798  ...          0            0          0
499996  93031488       53      1785  ...          0            0          0
499997  93031587       29      1540  ...          0            0          0
499998  93032161       93      1749  ...          0            0          0
499999  93032179       29      1503  ...          0            0          0

[500000 rows x 21 columns]


In [0]:
data.columns.values

array(['ad_id', 'make_id', 'model_id', 'fuel_type_id', 'body_type_id',
       'source_id', 'state_id', 'badge', 'year', 'km_driven', 'image',
       'url', 'price', 'title', 'date_time', 'hash', 'coordinates',
       'city', 'duplicate', 'click_count', 'ad_status'], dtype=object)

# **Transforming "badge" to "badge_transformed"**

In [0]:
#removing NaNs
data[['badge']] = data[['badge']].fillna("")

In [0]:
#replacing 'no' and 'all others' to empty string in badge column
count = data.badge.str.contains('(?i)(^no$)|(^all\W?others$)', regex=True).sum()
print("count:",count)
print(data[data.badge.str.contains('(?i)(^no$)|(^all\W?others$)', regex=True)]['badge'][0:5])
data['badge_transformed'] = data.badge.replace(to_replace=r'(?i)(^no$)|(^all\W?others$)', value='', regex=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


count: 22956
2     no
5     no
6     no
20    no
22    no
Name: badge, dtype: object


In [0]:
#numeric badges
count = data.badge_transformed.str.contains('^[0-9]+$', regex=True).sum()
print("count:",count)
print(data[data.badge_transformed.str.contains('^[0-9]+$', regex=True)]['badge_transformed'])

count: 2096
309        24
485        16
2040       32
2258      500
2905      350
         ... 
498902    110
499263    110
499311      5
499525    110
499600     90
Name: badge_transformed, Length: 2096, dtype: object


In [0]:
#replacing '+' and all other garbage non-character entries
count = data.badge_transformed.str.contains('^[^0-9A-Za-z]+$', regex=True).sum()
print("count:",count)
print(data[data.badge_transformed.str.contains('^[^0-9A-Za-z]+$', regex=True)]['badge_transformed'][0:5])
data['badge_transformed'] = data.badge_transformed.replace(to_replace=r'^[^0-9A-Za-z]+$', value='', regex=True)

count: 388
993     +
1981    +
3099    +
6482    +
6816    +
Name: badge_transformed, dtype: object


In [0]:
#standardizing badge representations
data['badge_transformed'] = data.badge_transformed.str.upper()

data['badge_transformed'] = data.badge_transformed.replace(to_replace=r'-| ', value='', regex=True)

count = data.badge_transformed.str.contains('(?i)4WD', regex=True).sum()
print("count:",count)
print(data[data.badge_transformed.str.contains('(?i)4WD', regex=True)]['badge_transformed'][0:5])
data['badge_transformed'] = data.badge_transformed.replace(to_replace=r'(?i)4WD', value='4X4', regex=True)

count: 2344
345               4WD
1028      TITANIUM4WD
1208            ST4WD
1389    TREND4WD7SEAT
1411     STL4WDNSPORT
Name: badge_transformed, dtype: object


In [0]:
#removing 'lpg' from badge as that is already recorded in the fuel_type column
count = data.badge_transformed.str.contains('(?i)lpg', regex=True).sum()
print("count:",count)
print(data[data.badge_transformed.str.contains('(?i)lpg', regex=True)]['badge_transformed'][0:5])
data['badge_transformed'] = data.badge_transformed.replace(to_replace=r'(?i)lpg', value='', regex=True)

count: 1064
41              G6LPG
790             XTLPG
971          OMEGALPG
1445    SV6ZSERIESLPG
1476         OMEGALPG
Name: badge_transformed, dtype: object


In [0]:
data.badge.head()

0    103-TDI-COMFORTLINE
1                 ST-4x4
2                     no
3             XLT-32-4x4
4                     SX
Name: badge, dtype: object

In [0]:
data.badge_transformed.head()

0    103TDICOMFORTLINE
1                ST4X4
2                     
3             XLT324X4
4                   SX
Name: badge_transformed, dtype: object

In [0]:
data.badge.tail()

499995           SRT-8
499996            SRT8
499997          CV-2WD
499998         F-SPORT
499999    ASCENT-SPORT
Name: badge, dtype: object

In [0]:
data.badge_transformed.tail()

499995           SRT8
499996           SRT8
499997          CV2WD
499998         FSPORT
499999    ASCENTSPORT
Name: badge_transformed, dtype: object

In [0]:
#data.to_csv("drive/My Drive/FYP/Data/ads_fact_500000_badge.csv", encoding='utf-8', index=False)

# **Transforming "coordinates" to "cities" and "states"**

In [0]:
# auspost_locations = pd.read_csv("drive/My Drive/FYP/Data/auspost-locations.csv", usecols=['Suburb','State','Lat','Lon'], engine='python', encoding='utf-8', error_bad_lines=False)
# auspost_locations.head()

In [0]:
# suburbs = []
# states = []
# for i in range(0, len(data.index)):
#     print(i)
#     try:
#         coord1=tuple(np.array(data.iloc[i].coordinates.split(',')).astype(np.float))
#     except:
#         continue
#     min_dist = math.inf
#     min_index = 0
#     for j in range(0, len(auspost_locations.index)):
#         coord2=(auspost_locations.iloc[j].Lat, auspost_locations.iloc[j].Lon)
#         dist = distance.distance(coord1,coord2)
#         if dist < min_dist:
#             min_dist = dist
#             min_index = j

#     suburbs.append(auspost_locations.iloc[min_index].loc['Suburb'])
#     states.append(auspost_locations.iloc[min_index].loc['State'])

In [0]:
# np.savetxt('drive/My Drive/FYP/Data/suburbs.txt', suburbs, delimiter=',')
# np.savetxt('drive/My Drive/FYP/Data/states.txt', states, delimiter=',')

In [0]:
state_files = os.listdir("drive/My Drive/FYP/Data/Location/")
city2state = {}
for file in state_files:
    with open("drive/My Drive/FYP/Data/Location/"+file) as f:
        cities = [i.strip() for i in f.read().split("\n")]
        for city in cities:
          city2state[city] = file.replace(".txt", "")
print(city2state)

In [0]:
states = []
cities = []
for i in range(0, len(data.index)):
  try:
    coordinates = tuple(np.array(data.iloc[i].coordinates.split(',')).astype(np.float)),
    try:
      city = reverse_geocode.search(coordinates)[0]['city']
    except:
      city = data.iloc[i].coordinates
  except:
    city = data.iloc[i].coordinates
  
  states.append(city2state[city])
  cities.append(city)

KeyError: ignored

In [0]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
city=[]
for i in range (data.size):
  location = geolocator.reverse(data.coordinates[i])
  #city.append(location.address.split(",")[-4].strip())
  #print (i, location.address.split(",")[-4].strip())
  loc_dict = location.raw
  if 'city' in loc_dict['address']:
    print(i, loc_dict['address']['city'])

#location = geolocator.reverse("-31.7969755,115.8154526")

# location = geolocator.reverse("-37.815365,145.2265472")
# print (location.address)

0 Sydney
1 Sydney
2 Sydney
3 Perth
6 Joondalup
7 Perth
8 Perth
13 Toowoomba
15 Geelong


In [0]:
data['city2state'] = states
data['coordinates2city'] = cities
data.coordinates2city
data.city2state

ValueError: ignored

In [0]:
print(data.columns.values)
print(data)

array(['ad_id', 'make_id', 'model_id', 'fuel_type_id', 'body_type_id',
       'source_id', 'state_id', 'badge', 'year', 'km_driven', 'image',
       'url', 'price', 'title', 'date_time', 'hash', 'coordinates',
       'city', 'duplicate', 'click_count', 'ad_status',
       'badge_transformed', 'coordinates2city'], dtype=object)

In [0]:
data.coordinates2city

0                 Gymea
1           Castle Hill
2            Toongabbie
3             Joondanna
4           Forestville
              ...      
499995           Myaree
499996           Myaree
499997      Morningside
499998      Castle Hill
499999    Mayfield West
Name: coordinates2city, Length: 500000, dtype: object

{'Canberra': 'ACT', 'Adelaide': 'SA', 'Ceduna': 'SA', 'Clare': 'SA', 'Coober Pedy': 'SA', 'Gawler': 'SA', 'Goolwa': 'SA', 'Iron Knob': 'SA', 'Leigh Creek': 'SA', 'Loxton': 'SA', 'Millicent': 'SA', 'Mount Gambier': 'SA', 'Murray Bridge': 'SA', 'Naracoorte': 'SA', 'Oodnadatta': 'SA', 'Port Adelaide Enfield': 'SA', 'Port Augusta': 'SA', 'Port Lincoln': 'SA', 'Port Pirie': 'SA', 'Renmark': 'SA', 'Victor Harbor': 'SA', 'Whyalla': 'SA', 'Broome': 'WA', 'Bunbury': 'WA', 'Busselton': 'WA', 'Coolgardie': 'WA', 'Dampier': 'WA', 'Derby': 'WA', 'Fremantle': 'WA', 'Geraldton': 'WA', 'Kalgoorlie': 'WA', 'Kambalda': 'WA', 'Katanning': 'WA', 'Kwinana': 'WA', 'Mandurah': 'WA', 'Meekatharra': 'WA', 'Mount Barker': 'WA', 'Narrogin': 'WA', 'Newman': 'WA', 'Northam': 'WA', 'Perth': 'WA', 'Port Hedland': 'WA', 'Tom Price': 'WA', 'Wyndham': 'WA', 'Albury-Wodonga': 'Vic', 'Armidale': 'NSW', 'Ballina': 'NSW', 'Balranald': 'NSW', 'Batemans Bay': 'NSW', 'Bathurst': 'NSW', 'Bega': 'NSW', 'Bourke': 'NSW', 'Bowral'

In [0]:
#data.to_csv("drive/My Drive/FYP/Data/ads_fact_500000_badge_location.csv", encoding='utf-8', index=False)

# **Other transformations**

In [0]:
data['datetime_year'] = 0
data['datetime_month'] = 0
data['datetime_daytime'] = ''

data[['datetime_year','datetime_month','datetime_daytime']] = data.date_time.str.split('-', expand=True)

data = data.drop(['datetime_daytime'], axis=1)
data['datetime_year'] = data['datetime_year'].astype(int)
data['datetime_month'] = data['datetime_month'].astype(int)

print(data.datetime_year)
print(data.datetime_month)

0         2019
1         2019
2         2019
3         2019
4         2019
          ... 
499995    2019
499996    2019
499997    2019
499998    2019
499999    2019
Name: datetime_year, Length: 500000, dtype: int64
0          7
1          7
2          7
3          7
4          7
          ..
499995    11
499996    11
499997    11
499998    11
499999    11
Name: datetime_month, Length: 500000, dtype: int64


In [0]:
print(data.columns.values)
print(len(data))
print(data)

['ad_id' 'make_id' 'model_id' 'fuel_type_id' 'body_type_id' 'source_id'
 'state_id' 'badge' 'year' 'km_driven' 'image' 'url' 'price' 'title'
 'date_time' 'hash' 'coordinates' 'city' 'duplicate' 'click_count'
 'ad_status' 'badge_transformed' 'coordinates2city' 'datetime_year'
 'datetime_month']
500000
           ad_id  make_id  ...  datetime_year  datetime_month
0       73919640       34  ...           2019               7
1       73919652        8  ...           2019               7
2       73919677        8  ...           2019               7
3       73919705       66  ...           2019               7
4       73919723       81  ...           2019               7
...          ...      ...  ...            ...             ...
499995  93031487       86  ...           2019              11
499996  93031488       53  ...           2019              11
499997  93031587       29  ...           2019              11
499998  93032161       93  ...           2019              11
499999  93032179

In [0]:
data.to_csv("drive/My Drive/FYP/Data/ads_fact_500000_transformed.csv", encoding='utf-8', index=False)